In [1]:
import os
import boto3
import random
from datetime import datetime
from typing import List, Dict, Any
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_aws import ChatBedrock

In [2]:
load_dotenv()

class Config:
    AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
    KB_ID = os.getenv("KB_ID")
    MODEL_ARN = os.getenv("MODEL_ARN")
    DYNAMODB_TABLE_NAME = os.getenv("DYNAMODB_TABLE_NAME")
    AWS_REGION = os.getenv("AWS_REGION", "us-east-1")

# Initialize AWS Bedrock Runtime Client
client = boto3.client("bedrock-agent-runtime", region_name=Config.AWS_REGION)

In [ ]:
llm = ChatBedrock(
    model="amazon.nova-pro-v1:0",
    region_name=Config.AWS_REGION,
    beta_use_converse_api=True,
    streaming=True
)


In [14]:
from boto3.dynamodb.conditions import Key,Attr
dynamodb = boto3.resource('dynamodb',region_name=Config.AWS_REGION)
TABLE_NAME = Config.DYNAMODB_TABLE_NAME
table = dynamodb.Table(TABLE_NAME)

In [5]:
def retrieve_from_kb(input_text: str):
    if not isinstance(input_text, str):
        raise TypeError("Expected input_text to be a string")

    try:
        response = client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
    "type": "KNOWLEDGE_BASE",
    "knowledgeBaseConfiguration": {
        "knowledgeBaseId": Config.KB_ID,
        "modelArn":Config.MODEL_ARN,
        "retrievalConfiguration": {
            "vectorSearchConfiguration": {
                "numberOfResults": 5
            }
        },
        "generationConfiguration": {
            "promptTemplate": {
                "textPromptTemplate": "You are a precise HR assistant at Ayatacommerce that answers questions using ONLY the provided context and consider you as an employee of ayatacommerce responsible for responding to all queries related to the company by other employees.\r\n\r\nContext:\r\n$search_results$\r\n\r\nQuestion: $query$\r\n\r\nAlways respond in a concise and professional manner and also just don't answer simply if always create a scentence with the answer.\r\nNOTE:Do not mention the source of the context or the document name in your answer or anything related to the provided context.\r\nIf no information is avialable in the context, please respond with the following data:\r\nContact HR at Ayatacommerce for assistance: hr@ayatacommerce.com \r\nHuman Resourse email: hr@ayatacommerce.com\r\n\r\nRules:\r\n1. If the context contains relevant information, provide a concise answer based solely on that.\r\n2. If the question asks about something NOT in the context, respond ONLY with: 'I don't have this information in my knowledge base. Please contact hr@ayatacommerce.com for assistance.'\r\n3. Never infer or make up information not explicitly stated in the context.\r\n4. If the question is ambiguous or unclear, ask for clarification.\r\n5. Do not provide any personal opinions or subjective statements.\r\n6. Always maintain a professional tone and language.\r\n7. Avoid using filler phrases like 'I think' or 'In my opinion'.\r\n8. If the context is too long, summarize it before answering.\r\n10. If the question is a yes/no question, provide a clear yes or no answer based on the context.\r\n11. If the question is a list, provide a clear and concise list based on the context.\r\n12. If the question is a how-to question, provide a clear and concise step-by-step guide based on the context.\r\n13. If the question is a why question, provide a clear and concise explanation based on the context.\r\n14. If the question is a when question, provide a clear and concise answer based on the context.\r\n\r\nAnswer:\"\"\""
            },
            "inferenceConfig": {
                "textInferenceConfig": {
                    "temperature": 0,
                    "topP": 0.9,
                    "maxTokens": 512,
                    "stopSequences": []
                }
            }
        },
        "orchestrationConfiguration": {
            "inferenceConfig": {
                "textInferenceConfig": {
                    "temperature": 0,
                    "topP": 0.9,
                    "maxTokens": 512,
                    "stopSequences": []
                }
            }
        }
    }
}
        )
        return response
    except Exception as e:
        print(f"Error retrieving knowledge base: {e}")
        return None


In [6]:
def rewrite_question(user_input: str, chat_history: List) -> str:
    messages = []
    
    # Ensure we start with a user message
    if not chat_history or isinstance(chat_history[0], HumanMessage):
        messages.append(HumanMessage(content="You are a helpful assistant that rewrites questions to be standalone."))
    
    # Add conversation history (alternating user/assistant messages)
    for entry in chat_history[-10:]:  # Keep only the latest 10 for context
        messages.append(entry)
    
    retriever_prompt = (
        "Given a chat history and the latest user question which might reference context in the chat history, "
        "formulate a standalone question which can be understood without the chat history. "
        "Do NOT answer the question, just reformulate it if needed and otherwise return it as is. "
        "Latest question: {question}"
    )
    
    messages.append(HumanMessage(content=retriever_prompt.format(question=user_input)))
    
    try:
        response = llm.invoke(messages)
        return response.content if isinstance(response, AIMessage) else str(response)
    except Exception as e:
        print(f"Error rewriting question: {e}")
        return user_input  # Fallback to original input

In [84]:
class ChatHistoryManager:
    def __init__(self, user_id: str = "default", session_id: str = "default_session"):
        self.user_id = user_id
        self.session_id = session_id

    def load_history(self) -> Dict[str, Any]:
        try:
            response = table.get_item(
                Key={
                    'user_id': self.user_id,
                    'session_id': self.session_id
                }
            )
            return response.get('Item', {
                'user_id': self.user_id,
                'session_id': self.session_id,
                'messages': []
            }) 
        except Exception as e:
            print("Error loading from DynamoDB:", e)
            return {
                'user_id': self.user_id,
                'session_id': self.session_id,
                'messages': []
            }
            
    def save_history(self, history: Dict[str, Any]):
        try:
            table.put_item(Item=history)
        except Exception as e:
            print("Error writing to DynamoDB:", e)

    def summarize_if_needed(self, history: Dict[str, Any]) -> Dict[str, Any]:
        messages = history['messages']
        if len(messages) < 5:
            return history

        # Get the last 10 unsummarized messages
        unsummarized_messages = messages[-10:]
        
        history_text = "\n".join(
            f"User: {msg['user']['content']}\nAssistant: {msg['assistant']['content']}"
            for msg in unsummarized_messages
        )

        summary_prompt = f"""
        Summarize the following 10 interactions into one concise but informative summary:
        {history_text}
        """

        summary = llm.invoke(summary_prompt.strip())
        if isinstance(summary, AIMessage):
            summary = summary.content

        # Create a summary message
        summary_message = {
            "user": {
                "content": summary,
                "timestamp": datetime.now().isoformat(),
                "is_summary": True
            },
            "assistant": {
                "content": "This is a summary of previous interactions",
                "timestamp": datetime.now().isoformat(),
                "is_summary": True
            }
        }

        # Keep only messages that weren't summarized
        history['messages'] = messages[:-10] + [summary_message]
        return history

    def append_chat_pair(self, history: Dict[str, Any], user_msg: str, assistant_msg: str) -> Dict[str, Any]:
        new_entry = {
            "user": {
                "content": user_msg,
                "timestamp": datetime.now().isoformat()
            },
            "assistant": {
                "content": assistant_msg,
                "timestamp": datetime.now().isoformat()
            }
        }

        history['messages'].append(new_entry)
        self.save_history(history)
        return self.summarize_if_needed(history)

In [ ]:
def is_history_within_token_limit(user_id, session_id) -> bool:
    try:
        history_data = ChatHistoryManager(user_id, session_id).load_history()

        messages = (
            message.get('user', {}).get('content', '') + ' ' +
            message.get('assistant', {}).get('content', '')
            for message in history_data.get('messages', [])
        )
        combined_text = ' '.join(messages)
        total_tokens = llm.get_num_tokens(combined_text)
        return total_tokens < 280000

    except Exception as e:
        print(f"[Token Count Error] User: {user_id}, Session: {session_id}, Error: {e}")
        return False


In [41]:
def chat(user_input: str, user_id: str = "default", session_id: str = "default_session") -> str:
    history_manager = ChatHistoryManager(user_id, session_id)
    chat_history = history_manager.load_history()

    try:
        # Format history for context
        formatted_history = []
        for entry in chat_history['messages']:
            user_content = entry['user']['content']
            assistant_content = entry['assistant']['content']
            
            if entry['user'].get('is_summary', False):
                formatted_history.append(AIMessage(content=user_content))
            else:
                formatted_history.append(HumanMessage(content=user_content))
                formatted_history.append(AIMessage(content=assistant_content))

        # Rewrite the question to be standalone
        # print(formatted_history)
        standalone_question = rewrite_question(user_input, formatted_history)
        print(f"Standalone question: {standalone_question[0]['text']}")

        # Call retrieve_and_generate with clean string
        response = retrieve_from_kb(standalone_question[0]['text'])
        if not response:
            raise ValueError("No response returned from retrieve_from_kb")

        # Handle response format
        if isinstance(response, dict) and 'output' in response:
            answer = response['output'].get('text', 'Sorry, I could not find information about that topic.')
        else:
            answer = 'Sorry, I could not find information about that topic.'

        # Save chat
        chat_history = history_manager.append_chat_pair(
            chat_history, user_msg=user_input, assistant_msg=answer
        )

        return answer

    except Exception as e:
        print(f"Error in RAG chain: {e}")
        fallback_responses = [
            "I'm having trouble accessing that information. Could you rephrase your question?",
            "My knowledge base seems to be unavailable at the moment. Please try again later.",
            "I encountered an unexpected error while processing your request."
        ]
        return random.choice(fallback_responses)

In [42]:
chat("what do you know about the CEO of AyataCommerce?","123","1234")

{'Item': {'messages': [{'user': {'content': 'what do you know about the CEO of AyataCommerce?', 'timestamp': '2025-06-02T16:45:08.926719'}, 'assistant': {'content': 'The CEO of AyataCommerce is Shine Mathew, who is the Founder of the company. He welcomes everyone to AyataCommerce and shares what it means to work at the company.', 'timestamp': '2025-06-02T16:45:08.926719'}}, {'user': {'content': 'Tell me about this company', 'timestamp': '2025-06-02T16:45:32.139576'}, 'assistant': {'content': 'AyataCommerce is a company that focuses on providing structure for its clients and a framework to understand the core issues they are solving. The CEO and Founder of AyataCommerce is Shine Mathew, who welcomes everyone to the company and emphasizes the values of empathy, inclusion, diversity, and respect. Employees describe AyataCommerce as a place where open communication, inclusion, and work-life balance are valued. The company encourages collaboration, professional growth, and flexible working 

'The context provides a welcoming message from Shine Mathew, the CEO and Founder of AyataCommerce, but does not offer detailed information about his background, achievements, or specific contributions to the company. For more detailed information, please contact HR at Ayatacommerce: hr@ayatacommerce.com.'

In [34]:
chat("Tell me about this company","123","1234")

Standalone question: Could you provide information about the company AyataCommerce, including details about its CEO, Shine Mathew, and what it is like to work there?


'AyataCommerce is a company that focuses on providing structure for its clients and a framework to understand the core issues they are solving. The CEO and Founder of AyataCommerce is Shine Mathew, who welcomes everyone to the company and emphasizes the values of empathy, inclusion, diversity, and respect. Employees describe AyataCommerce as a place where open communication, inclusion, and work-life balance are valued. The company encourages collaboration, professional growth, and flexible working arrangements. Microsoft Teams is the primary communication channel used within the organization.'

In [10]:
print(chat("what do you know about the CEO of AyataCommerce?", "123", "1234"))

Error loading from DynamoDB: An error occurred (ValidationException) when calling the Query operation: Query key condition not supported
Standalone question: What information is available about the CEO of AyataCommerce?
The CEO of AyataCommerce is Shine Mathew. He is the founder of the company and has welcomed everyone to AyataCommerce, highlighting what it means to work there.


In [11]:
print(chat("what do you know about him", "123", "1234"))

Error loading from DynamoDB: An error occurred (ValidationException) when calling the Query operation: Query key condition not supported
Standalone question: Who is the person you are inquiring about, and what specific information are you interested in knowing about them?
I don't have this information in my knowledge base. Please contact hr@ayatacommerce.com for assistance.


In [35]:
print(chat("What is the net worth of this company?", "1234", "1234"))


Standalone question: What is the current net worth of the specified company?
I don't have this information in my knowledge base. Please contact hr@ayatacommerce.com for assistance.


In [37]:
print(chat("what do you know about him", "123", "1234"))


Standalone question: What do you know about Shine Mathew, the CEO of AyataCommerce?
Shine Mathew is the CEO and Founder of AyataCommerce. He welcomes everyone to AyataCommerce and emphasizes the meaning of working at Ayata.
